## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
bank_data = pd.read_csv('/datasets/data.csv')
bank_data.info() # Информация о таблице
bank_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В данной таблице встречаются некоторые артефакты:
- Отрицательные значения в колонке "days_employed"
- Пропуски данных в колонках "days_employed" и "total_income"
- В колонке "gender" одна строка имеет значение "XNA"

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
bank_data.isna().sum() # Проверяем в каких столбцах есть пропуски данных

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
bank_data['days_employed'] = bank_data['days_employed'].fillna(0) # Приводим пропуски в столбце 'days_employed' 
                                                                    # к значению ноль 

In [4]:
total_income_median = bank_data['total_income'].median()  # Медианное значение ежемесячного дохода 
bank_data['total_income'] = bank_data['total_income'].fillna(total_income_median) # Приводим пропуски в столбце 'total_income'
                                                                                  #  к медианному значению месячного дохода

In [5]:
bank_data.isna().sum() #Финальная проверка на пропуски

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [6]:
bank_data[bank_data['dob_years'] == 0].count() #101 строка с возрастом равным нулю.

children            101
days_employed       101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income        101
purpose             101
dtype: int64

In [7]:
dob_year_means = bank_data['dob_years'].mean() #среднее значение возраста заемщиков
bank_data['dob_years'] = bank_data['dob_years'].replace(0, dob_year_means)  #заменяем нулевые значения возраста 
                                                                            #на среднее значение

In [8]:
bank_data[bank_data['gender'] == 'XNA']
bank_data.drop([10701], inplace=True)  #удаляем строку с ошибочным значением. 10701 - номер строки в bank_data

In [9]:
bank_data['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [10]:
bank_data['children'] = bank_data['children'].replace(-1, 1)

In [11]:
bank_data['children'] = bank_data['children'].replace(20, 2)

In [12]:
bank_data['children'].value_counts()

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

Пропущенные значения заполнены. 

Пропущенные значения в столбцах "days_employed" и "total_income" находятся в одних и тех же сроках. Появление пропусков могло
произайти по нескольким причинам:
1. Пропущенные данные не были предоставлены банку.
2. Во время выгрузки данных возникла ошибка. 

В столбце "dob_years" были строки с нулевым значением. Заменил их на среднее значение возраста.

### Замена типа данных

In [13]:
bank_data.info() # проверяем типы данных 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


In [14]:
bank_data['dob_years'] = bank_data['dob_years'].astype(int)
bank_data['days_employed'] = bank_data['days_employed'].astype(int)
bank_data['total_income'] = bank_data['total_income'].astype(int)

In [15]:
bank_data.info() # проверяем, изенились ли типы данных

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод 


Заменил тип данных с float на integer в трех столбцах: 'dob_years', 'days_employed' и 'total_income'. Замена типа данных произведена методом astype() 

В столбце 'dob_years' тип данных float появился после присвоения пропущенным значениям медианных.

### Обработка дубликатов

In [16]:
bank_data['education'].value_counts() # проверяем столбец 'education' на предмет дубликатов.

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [17]:
bank_data['education'] = bank_data['education'].str.lower() # приводим значения в столбце 'education' к нижниму регистру

In [18]:
bank_data['education'].value_counts() 

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

В столбце 'education' были дубликаты. Это произошло из-за тогда что значения были написаны в верхнем и нижнем регистрах.
Убрал дубликаты путем приведения значений к нижнему регистру.

In [19]:
bank_data.duplicated().sum() # проверка данных на дубликаты.
bank_data = bank_data.drop_duplicates().reset_index(drop=True)

In [20]:
bank_data.duplicated().sum()

0

### Вывод

#### В данных было несколько типов дубликатов.
Дубликаты в столбце "edication" появились из-за написания символов в разных регистрах. Эти дубликаты были удалены методом .str.lower()

Так же в данных были полные дубликаты строк в колчестве 71 шт. От них избавился методом drop_duplicates(). Эти дубликаты могли появиться при выгрузке данных.

### Лемматизация

In [21]:
from pymystem3 import Mystem # вызываем библиотеку pymystem3 для лемматизации
m = Mystem()

bank_data['purpose'] = bank_data['purpose'].str.lower()
purpose_dict = bank_data['purpose'].unique() # сохраняем в переменной уникальные значения с целями получения кредита.
purpose_dict_join = ' '.join(purpose_dict) # методом .join() склеиваем список в строку




In [22]:
purpose_lemmas = m.lemmatize(purpose_dict_join) # лемматизируем уникальные значения с целями получения кредита.
print(purpose_lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

In [23]:
from collections import Counter # посчтиаем количество уникальных лемм. 
                                # Для этого используем контейнер Counter из модуля collections.
print(Counter(purpose_lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


In [24]:
bank_data['purpose_lemmas'] = bank_data['purpose'].apply(m.lemmatize) # методом .apply() добавляем в таблицу новый столбец с 
                                                                      # с леммами.
bank_data.head()  # проверка работы метода.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


### Вывод

1. Здесь я выделил леммы для уникальных значений с целями получения кредита. Так же подсчитал количество уникальных лемм.
    Наиболее часто встречающиеся: это покупка автомобиля или недвижимости, а так же образование.
    
2. В таблицу добавлен новый столбец с леммами целей кредитования

### Категоризация данных

In [25]:
def purpose_category(row):                          # Создаем функцию для категоризации целей кредитования
    for i in row['purpose_lemmas']:                 # на вход функция получает лемматизированное значение целей кредитования
        if 'жилье' in i or  'недвижимость' in i:    # а возвращает названние категории.
            return 'операции с недвижимостью'
        if 'автомобиль' in i:
            return 'приобретение автомобиля'
        if 'образование' in i:
            return 'образование'
        if 'свадьба' in i:
            return 'свадьба'
    else:
        return 'другое'
        
bank_data['purpose_category'] = bank_data.apply(purpose_category, axis=1)
# чтобы создать отдельный столбец с категориями вызовем метод .apply()

bank_data['purpose_category'].value_counts()  # Выведем статистику по категориям целей кредитования

операции с недвижимостью    10810
приобретение автомобиля      4306
образование                  4013
свадьба                      2324
Name: purpose_category, dtype: int64

In [26]:
bank_data[bank_data['total_income'] < 100000]['total_income'].count() 
#число людей с доходом ниже 100 тысяч

4463

In [27]:
bank_data[(bank_data['total_income'] > 100000) & (bank_data['total_income'] < 200000)]['total_income'].count() 
#число людей с доходом от 100 тысяч до 200 тысяч

11925

In [28]:
bank_data[bank_data['total_income'] > 200000]['total_income'].count()
#число людей с доходом выше 200 тысяч

5065

На основе данных таблицы из таблицы видно, что наибольшая часть (~ 50%) людей имеет доход в диапзоне от 100 тысяч до 200.
Назовем эту группу - люди со `средним доходом`.

Остальные 50% тоже делятся поровну (~ 25%):
- При доходе меньше 100 тысяч - люди с `низким доходом`.
- При доходе больше 200 тысяч - люди с `высоким доходом`.

In [29]:
def income_level(total_income):   # Создаем функцию для категоризации данных о доходе.
    if total_income <= 100000:     # На вход функция получает суммарный доход, а возвращает она категорию уровня дохода.
        return 'Низкий доход'
    if 100000 < total_income <= 200000:
        return 'Средний доход'
    else:
        return 'Высокий доход'
        
bank_data['income_level'] = bank_data['total_income'].apply(income_level)
#создаем новый столбец методом .apply()

bank_data['income_level'].value_counts()  # Выведем статистику по уровню дохода.

Средний доход    11925
Высокий доход     5065
Низкий доход      4463
Name: income_level, dtype: int64

In [30]:
def family_type(children):      # Создаем функцию для категоризации данных о количестве детей.
    if children == 0:           # На вход функция получает количество детей, а возвращает она категорию семьи.
        return 'Семья без детей'
    if 0 < children <= 2:
        return 'Семья с детьми'
    else:
        return 'Многодетная семья'
    
bank_data['family_type'] = bank_data['children'].apply(family_type)
#создаем новый столбец методом .apply()

bank_data['family_type'].value_counts() # Выведем статистику по категории семьи.

Семья без детей      14090
Семья с детьми        6983
Многодетная семья      380
Name: family_type, dtype: int64

In [31]:
def debt_type(debt):  # Создаем функцию для категоризации данных о наличии задолжностей.
    if debt == 0:      # На вход функция получает значение 0 или 1, а возвращает она информацию о наличии задолжности.
        return 'Не было задолжности'   # 0 - не было задолжностей. 1 - была задолжность.
    else:
        return 'Была задолжность'
    
bank_data['debt_type'] = bank_data['debt'].apply(debt_type)
#создаем новый столбец методом .apply()

bank_data['debt_type'].value_counts() # Выведем статистику по наличию задолжности.

Не было задолжности    19712
Была задолжность        1741
Name: debt_type, dtype: int64

### Вывод

В этой части я категоризировал данные таблицы по 4 значениям:
- леммы целей кредитования
- данные о доходе
- количество детей
- наличие задолжности


В таблицу были добавлены 4 соответсвующих столбца с категориями исходных данных

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
data_pivot_family_type = bank_data.pivot_table(index=['family_type'], columns = 'debt_type', 
                                            values = 'children', aggfunc = 'count')
# для нахождения зависимости сделаем сводную таблицу методом .pivot_table()
data_pivot_family_type
# выведем таблицу на экран

debt_type,Была задолжность,Не было задолжности
family_type,,
Многодетная семья,31,349
Семья без детей,1063,13027
Семья с детьми,647,6336


In [33]:
data_pivot_family_type['debt_ratio'] = (data_pivot_family_type['Была задолжность'] / data_pivot_family_type['Не было задолжности']) 
# добавим в таблицу столбец 'debt_ratio'. Его значение это отношение заемщиков с задолжностью к заемщикам без задолжности
# распределенное по количеству детей.

data_pivot_family_type 
# выведем таблицу на экран.

debt_type,Была задолжность,Не было задолжности,debt_ratio
family_type,,,
Многодетная семья,31,349,0.088825
Семья без детей,1063,13027,0.081600
Семья с детьми,647,6336,0.102115


### Вывод

Изходя из данных сводной таблицы, можно сделать вывод, что люди с 1-2 детьми имеют больший процент 
людей просрочивших платеж. Это может проиходить из-за того, что у этих людей есть расходы по уходу за ребенком. 

Многодетные семьи можно не брать в расчет, так как их количество в разы меньше.




- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
data_pivot_family_status = bank_data.pivot_table(index=['family_status'], columns = 'debt_type', 
                                            values = 'family_status_id', aggfunc = 'count')
# для нахождения зависимости сделаем сводную таблицу методом .pivot_table()
data_pivot_family_status
# выведем таблицу на экран

debt_type,Была задолжность,Не было задолжности
family_status,,
Не женат / не замужем,274,2536
в разводе,85,1110
вдовец / вдова,63,896
гражданский брак,388,3762
женат / замужем,931,11408


In [35]:
data_pivot_family_status['debt_ratio'] = data_pivot_family_status['Была задолжность'] / data_pivot_family_status['Не было задолжности']
# добавим в таблицу столбец 'debt_ratio'. Его значение это отношение заемщиков с задолжностью к заемщикам без задолжности
# распределенное по семейному статусу.

data_pivot_family_status
# выведем таблицу на экран.

debt_type,Была задолжность,Не было задолжности,debt_ratio
family_status,,,
Не женат / не замужем,274,2536,0.108044
в разводе,85,1110,0.076577
вдовец / вдова,63,896,0.070312
гражданский брак,388,3762,0.103137
женат / замужем,931,11408,0.081609


### Вывод

Исходя из данных таблицы наибольшее количество задолжностей в проценном соотношении было у людей, которые ниразу не регистрировали брак. В данной категории процент людей с просрочеными платежами по отношению к людям без просрочек состовляет - 10%. В других группах данный показатель состовляет - 7-8%.

Можно предположить, что семейный статус имеет среднее влияние на возврат кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
data_pivot_income_level = bank_data.pivot_table(index=['income_level'], columns = 'debt_type', 
                                            values = 'total_income', aggfunc = 'count')
# для нахождения зависимости сделаем сводную таблицу методом .pivot_table()
data_pivot_income_level
# выведем таблицу на экран

debt_type,Была задолжность,Не было задолжности
income_level,,
Высокий доход,358,4707
Низкий доход,354,4109
Средний доход,1029,10896


In [37]:
data_pivot_income_level['debt_ratio'] = data_pivot_income_level['Была задолжность'] / data_pivot_income_level['Не было задолжности']
# добавим в таблицу столбец 'debt_ratio'. Его значение это отношение заемщиков с задолжностью к заемщикам без задолжности
# распределенное по уровню дохода

data_pivot_income_level
# выведем таблицу на экран.

debt_type,Была задолжность,Не было задолжности,debt_ratio
income_level,,,
Высокий доход,358,4707,0.076057
Низкий доход,354,4109,0.086152
Средний доход,1029,10896,0.094438


### Вывод

У людей с высоким уровнем дохода наименьший процент просроченых задолжностей. 
Но нельзя утверждать, что уровень дохода имеет болшое влияние на возврат кредита в срок. Так как различия между людьми с разным доходом состовляют ~1%. 

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
data_pivot_purpose = bank_data.pivot_table(index=['purpose_category'], columns = 'debt_type', 
                                            values = 'debt', aggfunc = 'count')
# для нахождения зависимости сделаем сводную таблицу методом .pivot_table()
data_pivot_purpose
# выведем таблицу на экран

debt_type,Была задолжность,Не было задолжности
purpose_category,,
образование,370,3643
операции с недвижимостью,782,10028
приобретение автомобиля,403,3903
свадьба,186,2138


In [39]:
data_pivot_purpose['debt_ratio'] = data_pivot_purpose['Была задолжность'] / data_pivot_purpose['Не было задолжности']
# добавим в таблицу столбец 'debt_ratio'. Его значение это отношение заемщиков с задолжностью к заемщикам без задолжности
# распределенное по цели кредита

data_pivot_purpose
# выведем таблицу на экран.

debt_type,Была задолжность,Не было задолжности,debt_ratio
purpose_category,,,
образование,370,3643,0.101565
операции с недвижимостью,782,10028,0.077982
приобретение автомобиля,403,3903,0.103254
свадьба,186,2138,0.086997


### Вывод

Изходя из данных наименьший процент должников находится в категорих "Операции с недвижимостью". Это возможно связано с тем, 
что срок кредитования для операции с недвижимостью	выше, а риски остаться без денег ниже.
Напротив, приобритение автомобиля влечет за собой дополнительные расходы.

Из всего выше сказаного, можно сделать вывод, что цель кредита имеет среднее влияние на его возврат в срок.

### Шаг 4. Общий вывод

#### Целью данной работы является проверка клиентво банка на влияние семейнного положения, наличия детей, уровня дохода и целей кредитования на возврат кредита в срок. 

По окончании исследования были сделаны следующие гипотезы:
#### 1. Наличие, у заемщика, детей влияет на возврат кредита в срок. 
Наибольшый процент задолжности у заемщиков, имеющих 1-2 дейте. Он равен 10%. У других груп то показатель меньше на 2%. 
При наличии детей, расходы, в том числе непредвиденные, становятся намного больше.
    
#### 2. Уровень дохода не имеет большого влияние на возврат кредита в срок.
Это подтверждается тем, что процент задолжности у заемщиков, имеющих разный уровень дохода, практически одинаковый. Наиболее вероятно, что такая ситуация возникает из-за того, что люди с разным уровнем дохода берут кредит на разные суммы.

#### 3. Семейный статус имеет влияние на возврат кредита в срок.
Процент задолжности у не женатых/не замужних или живущих граждански браком заёмщиков состовляет 10%. Это на 2% больше чем у людей в браке. Скорее всего это связано с тем, что люди в браке имеют дополнительный доход со стороны своего партнера и реже допускают просрочки по платежам.

#### 4. Цель кредита может влиять на возврат кредита в срок.
Наибольший процент задолжности у заемщиков берущих кредит на приобретение автомобиля и на образование. Он состовляет 10%. В отличие от заемщиков берущих кредит на приобретение жилья (7.8%) или проведение свадьбы (8.6%). Такое распределение связано с тем, что приобретение автомобиля и образование не приносят прибыли, а наоборот влекут за собой дополнительные расходы (автомобиль) или уменьшение прибыли в краткосрочной перспективе (образование)









### Мои рекомендации: 

- Заемщики, которые вернут кредит в срок с наибольшей вероятностью это - люди находящиеся в браке, без детей, имеющие высокий доход, целю кредита которых, является покупка жилья.

- Заемщики, которые не вернут кредит в срок с наибольшей вероятностью это - Не женатые / замужние люди с 1-2 детьми, имеющие средний доход, целю кредита которых, является покупка автомобиля.